In [1]:
from jenga.tasks.openml import OpenMLTask
from jenga.cleaning.cleaners import AutoGluonCleaner
from jenga.corruptions.numerical import Scaling
from jenga.corruptions.generic import MissingValues
from jenga.evaluation.autocleaning import AutoCleaningEvaluator
import numpy as np

/Users/felix/code/yacl/yacl_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/felix/code/yacl/yacl_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/felix/code/yacl/yacl_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/felix/code/yacl/yacl_env/lib/python3.7/site-pack

In [2]:
task = OpenMLTask(0)

cleaner = AutoGluonCleaner(numeric_columns=task.numerical_columns, 
                           categorical_columns=task.categorical_columns)

In [ ]:
model = task.fit_baseline_model(task.train_data, task.train_labels)
cleaner.fit(task.train_data)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    1.6s finished
Traceback (most recent call last):
  File "/Users/felix/code/yacl/yacl_env/lib/python3.7/site-packages/autogluon/utils/tabular/ml/trainer/abstract_trainer.py", line 261, in train_and_save
    model = self.train_single(X_train, y_train, X_test, y_test, model, kfolds=kfolds, k_fold_start=k_fold_start, k_fold_end=k_fold_end, n_repeats=n_repeats, n_repeat_start=n_repeat_start, level=level, time_limit=time_limit)
  File "/Users/felix/code/yacl/yacl_env/lib/python3.7/site-packages/autogluon/utils/tabular/ml/trainer/abstract_trainer.py", line 246, in train_single
    model.fit(X_train=X_train, Y_train=y_train, X_test=X_test, Y_test=y_test, time_limit=time_limit, **model_fit_kwargs)
  File "/Users/felix/code/yacl/yacl_env/lib/python3.7/site-packages/autogluon/utils/tabular/ml/models/tabular_nn/tabular_nn_model.py", line 185, in fit
    train_dataset

Traceback (most recent call last):
  File "/Users/felix/code/yacl/yacl_env/lib/python3.7/site-packages/autogluon/utils/tabular/ml/trainer/abstract_trainer.py", line 261, in train_and_save
    model = self.train_single(X_train, y_train, X_test, y_test, model, kfolds=kfolds, k_fold_start=k_fold_start, k_fold_end=k_fold_end, n_repeats=n_repeats, n_repeat_start=n_repeat_start, level=level, time_limit=time_limit)
  File "/Users/felix/code/yacl/yacl_env/lib/python3.7/site-packages/autogluon/utils/tabular/ml/trainer/abstract_trainer.py", line 246, in train_single
    model.fit(X_train=X_train, Y_train=y_train, X_test=X_test, Y_test=y_test, time_limit=time_limit, **model_fit_kwargs)
  File "/Users/felix/code/yacl/yacl_env/lib/python3.7/site-packages/autogluon/utils/tabular/ml/models/tabular_nn/tabular_nn_model.py", line 185, in fit
    train_dataset, test_dataset = self.generate_datasets(X_train=X_train, y_train=Y_train, params=params, X_test=X_test, y_test=Y_test)
  File "/Users/felix/code/ya

Traceback (most recent call last):
  File "/Users/felix/code/yacl/yacl_env/lib/python3.7/site-packages/autogluon/utils/tabular/ml/trainer/abstract_trainer.py", line 261, in train_and_save
    model = self.train_single(X_train, y_train, X_test, y_test, model, kfolds=kfolds, k_fold_start=k_fold_start, k_fold_end=k_fold_end, n_repeats=n_repeats, n_repeat_start=n_repeat_start, level=level, time_limit=time_limit)
  File "/Users/felix/code/yacl/yacl_env/lib/python3.7/site-packages/autogluon/utils/tabular/ml/trainer/abstract_trainer.py", line 246, in train_single
    model.fit(X_train=X_train, Y_train=y_train, X_test=X_test, Y_test=y_test, time_limit=time_limit, **model_fit_kwargs)
  File "/Users/felix/code/yacl/yacl_env/lib/python3.7/site-packages/autogluon/utils/tabular/ml/models/tabular_nn/tabular_nn_model.py", line 185, in fit
    train_dataset, test_dataset = self.generate_datasets(X_train=X_train, y_train=Y_train, params=params, X_test=X_test, y_test=Y_test)
  File "/Users/felix/code/ya

In [ ]:
evaluator = AutoCleaningEvaluator(task, cleaner)

In [ ]:
corruptions = []
for impacted_column in task.numerical_columns + task.categorical_columns:
    for fraction in [0.99, 0.5, 0.25, 0.1, 0.01]:
        if impacted_column in task.numerical_columns:
            corruptions.append(Scaling)
        for missingness in ['MCAR', 'MAR', 'MNAR']:
            corruption = MissingValues(impacted_column, fraction, missingness=missingness, na_value=np.nan)
            corruptions.append(corruption)

In [ ]:
evaluator.evaluate(model, 5, *corruptions[:2])